# Spider-Gender

## Fasi:
1. Estrazione dei dataset
2. Combinazione dei dataset

# Estrazione dei dataset
In questa fase vengono create le directory contenenti i dataset estratti.  
É necessario che nella cartella contenente i file si trovino i **.zip** dei 2 dataset.

In [1]:
#Inserire in una cartella di lavoro i file zip dei dataset
#Creazione delle directory ed estrazione dei dataset
import os
import utils as u

path=r"C:\Users\vince\Desktop\Progetto FVAB"
os.chdir(path)
cwd = os.getcwd()

#Crea directory per estrarre i dataset solo una volta all'inizio
u.makedir('celeba')
u.makedir('utkface')

#Estrae i due dataset solo una volta all'inizio
u.unzip('celeba-dataset.zip','celeba')
u.unzip('utkface-new.zip','utkface')

# Combinazione dei dataset
In questa fase sono combinati i due dataset e viene creato un indice che tiene traccia dei nuovi nomi 

| Old_Name    | New_Name   |
| :---: | :---: |
|   000001.jpg    | 000001_1.jpg     |
|  1_0_0_20161219140627985.jpg.chip.jpg  | 000002_0.jpg       |

Il nuovo nome è composto da:
* Parte sinistra un intero che incrementa di 1
* Parte destra dopo l'underscore che indica la label 
 0. Donna 
 1. Uomo

In [2]:
import utils as u
import os
path=r"C:\Users\vince\Desktop\Progetto FVAB"
os.chdir(path)

u.makedir('merged_dataset')
os.chdir('merged_dataset')
u.makedir('images')


path_images_celeba=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\img_align_celeba\img_align_celeba"
path_images_celeba2=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\img_align_celeba\img_align_celeba2"

path_images_utkface=r"C:\Users\vince\Desktop\Progetto FVAB\utkface\UTKFace"
path_images_utkface2=r"C:\Users\vince\Desktop\Progetto FVAB\\utkface\UTKFace2"

path_celeba_csv=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\list_attr_celeba.csv"
path_celeba_csv2=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\list_attr_celeba2.csv"

dest_path=r"C:\Users\vince\Desktop\Progetto FVAB\merged_dataset\images"
dest_path_oldnew_csv=r"C:\Users\vince\Desktop\Progetto FVAB\merged_dataset"

u.merge_datasets(path_images_celeba2,
                 path_images_utkface2,
                 path_celeba_csv2,
                 dest_path,
                 dest_path_oldnew_csv)


# Estrazione dei Landmarks
In questa fase utilizzo l'algoritmo fornito per ottenere i landmarks sul volto  

<img style="float:left;" src="web.PNG">

In [7]:
%%capture
os.chdir(path)
import calcolo_ragnatela as spider
spider.extract_landmarks_4circles_4sectors(r"C:\Users\vince\Desktop\Progetto FVAB\shape_predictor_68_face_landmarks.dat",r'C:\Users\vince\Desktop\Progetto FVAB\merged_dataset\images')

[[[194 231 253]
  [194 231 253]
  [194 231 253]
  ...
  [216 228 246]
  [223 237 255]
  [222 238 254]]

 [[194 231 253]
  [194 231 253]
  [194 231 253]
  ...
  [218 230 248]
  [223 237 255]
  [222 238 254]]

 [[194 231 253]
  [194 231 253]
  [194 231 253]
  ...
  [220 232 250]
  [224 238 255]
  [223 239 255]]

 ...

 [[ 26  74 140]
  [  1  49 115]
  [ 33  78 146]
  ...
  [ 28  55 122]
  [ 30  56 123]
  [ 30  56 122]]

 [[ 15  62 130]
  [ 23  70 138]
  [ 53  98 166]
  ...
  [ 20  49 118]
  [ 24  50 120]
  [ 24  51 118]]

 [[ 53 100 168]
  [ 89 136 204]
  [132 177 245]
  ...
  [ 20  49 118]
  [ 24  50 120]
  [ 24  50 120]]]
[[[ 45  52  67]
  [ 43  50  65]
  [ 43  50  65]
  ...
  [ 68  73  76]
  [ 74  79  82]
  [ 74  81  84]]

 [[ 45  52  67]
  [ 43  50  65]
  [ 43  50  65]
  ...
  [ 68  73  76]
  [ 74  79  82]
  [ 74  81  84]]

 [[ 47  52  67]
  [ 45  50  65]
  [ 43  50  65]
  ...
  [ 68  73  76]
  [ 74  79  82]
  [ 74  81  84]]

 ...

 [[221 211 211]
  [231 221 221]
  [229 221 221]
  ..

[[[ 50  61  69]
  [ 54  65  73]
  [ 59  72  80]
  ...
  [143 157 185]
  [144 158 186]
  [145 159 187]]

 [[ 50  61  69]
  [ 54  65  73]
  [ 59  72  80]
  ...
  [144 158 186]
  [144 158 186]
  [145 159 187]]

 [[ 50  61  69]
  [ 53  64  72]
  [ 58  71  79]
  ...
  [144 158 186]
  [145 159 187]
  [145 159 187]]

 ...

 [[ 27  42  51]
  [ 30  45  54]
  [ 35  50  59]
  ...
  [ 33  50  63]
  [ 35  54  69]
  [ 41  60  75]]

 [[ 28  43  52]
  [ 31  46  55]
  [ 36  51  60]
  ...
  [ 35  52  65]
  [ 36  55  70]
  [ 42  61  76]]

 [[ 28  43  52]
  [ 32  47  56]
  [ 36  51  60]
  ...
  [ 36  53  66]
  [ 37  56  71]
  [ 43  62  77]]]
[[[ 27  62 125]
  [ 29  64 127]
  [ 32  67 131]
  ...
  [ 49  73  97]
  [ 50  74  98]
  [ 51  75  99]]

 [[ 27  62 125]
  [ 28  63 126]
  [ 31  66 130]
  ...
  [ 48  72  96]
  [ 49  73  97]
  [ 50  74  98]]

 [[ 26  61 124]
  [ 28  63 126]
  [ 31  66 130]
  ...
  [ 46  72  96]
  [ 47  73  97]
  [ 48  74  98]]

 ...

 [[ 36  75 127]
  [ 33  72 124]
  [ 31  68 120]
  ..